# DataSets

## Sectors Dataset

El orden está en https://www.tradingview.com/markets/indices/quotes-snp/

In [1]:
import yfinance as yf

In [2]:
selected_stocks = ['^SP500-25','^SP500-30', '^SP500-35', '^SP500-20', '^SP500-45', '^SP500-15', '^SP500-60', '^SP500-50', '^SP500-55',
                  '^SP500-40', '^GSPE']
sp500_sectors = yf.download(selected_stocks,"2021-01-01","2021-11-30")

[*********************100%***********************]  11 of 11 completed


In [3]:
sp500_sectors['Adj Close']

,^GSPE,^SP500-15,^SP500-20,^SP500-25,^SP500-30,^SP500-35,^SP500-40,^SP500-45,^SP500-50,^SP500-55,^SP500-60
Date,,,,,,,,,,,
2021-01-04,286.500000,451.040009,730.440002,1287.849976,688.690002,1316.890015,483.799988,2250.169922,218.580002,310.760010,220.399994
2021-01-05,299.489990,461.420013,737.890015,1298.650024,688.760010,1325.670044,485.600006,2265.929932,219.169998,310.859985,220.220001
2021-01-06,308.450012,480.269989,755.340027,1305.209961,688.299988,1344.349976,506.779999,2225.020020,217.720001,318.510010,219.020004
2021-01-07,313.059998,484.040009,759.479980,1328.670044,686.070007,1362.089966,514.229980,2283.969971,220.770004,314.269989,219.690002
2021-01-08,312.779999,481.579987,757.789978,1352.530029,689.469971,1368.540039,513.239990,2301.169922,221.320007,317.000000,222.080002
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-22,419.380005,554.880005,894.799988,1664.420044,755.940002,1540.500000,657.080017,2971.409912,271.529999,342.649994,299.459991
2021-11-23,432.140015,555.369995,896.340027,1653.959961,761.200012,1541.859985,667.250000,2965.280029,270.450012,343.049988,302.739990
2021-11-24,436.369995,551.250000,895.200012,1656.619995,759.030029,1541.250000,665.729980,2986.290039,271.019989,342.829987,306.609985


In [4]:
sp500_sectors['Close']

,^GSPE,^SP500-15,^SP500-20,^SP500-25,^SP500-30,^SP500-35,^SP500-40,^SP500-45,^SP500-50,^SP500-55,^SP500-60
Date,,,,,,,,,,,
2021-01-04,286.500000,451.040009,730.440002,1287.849976,688.690002,1316.890015,483.799988,2250.169922,218.580002,310.760010,220.399994
2021-01-05,299.489990,461.420013,737.890015,1298.650024,688.760010,1325.670044,485.600006,2265.929932,219.169998,310.859985,220.220001
2021-01-06,308.450012,480.269989,755.340027,1305.209961,688.299988,1344.349976,506.779999,2225.020020,217.720001,318.510010,219.020004
2021-01-07,313.059998,484.040009,759.479980,1328.670044,686.070007,1362.089966,514.229980,2283.969971,220.770004,314.269989,219.690002
2021-01-08,312.779999,481.579987,757.789978,1352.530029,689.469971,1368.540039,513.239990,2301.169922,221.320007,317.000000,222.080002
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-22,419.380005,554.880005,894.799988,1664.420044,755.940002,1540.500000,657.080017,2971.409912,271.529999,342.649994,299.459991
2021-11-23,432.140015,555.369995,896.340027,1653.959961,761.200012,1541.859985,667.250000,2965.280029,270.450012,343.049988,302.739990
2021-11-24,436.369995,551.250000,895.200012,1656.619995,759.030029,1541.250000,665.729980,2986.290039,271.019989,342.829987,306.609985
